# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import LongType
#from pyspark.sql.functions import year, month, dayofmonth, hour
from pyspark.sql import Window
from pyspark.sql.functions import when, rand, mean, split, explode, max as maximum,min as minimum,datediff,from_unixtime, countDistinct as addupDistinct, sum as total, date_add

import datetime as dt

from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify app") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# load data
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [4]:
# check schema of data
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# check what types of pages exist
user_log.groupBy('page').count().orderBy(desc('count')).show(50)

+--------------------+------+
|                page| count|
+--------------------+------+
|            NextSong|228108|
|                Home| 14457|
|           Thumbs Up| 12551|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|         Roll Advert|  3933|
|               Login|  3241|
|              Logout|  3226|
|         Thumbs Down|  2546|
|           Downgrade|  2055|
|                Help|  1726|
|            Settings|  1514|
|               About|   924|
|             Upgrade|   499|
|       Save Settings|   310|
|               Error|   258|
|      Submit Upgrade|   159|
|    Submit Downgrade|    63|
|              Cancel|    52|
|Cancellation Conf...|    52|
|            Register|    18|
| Submit Registration|     5|
+--------------------+------+



In [6]:
# filter out null and empty ids
user_log = user_log.filter(user_log.userId != "")\
                    .filter(user_log.userId.isNotNull())\
                    .filter(user_log.sessionId.isNotNull())

In [7]:
# select particular columns to continue with
selected_data = user_log.select(['sessionId','userId','sessionId',\
                                'song','artist', 'gender','itemInSession',\
                                'length','level','location','page','status','ts','userAgent'])

In [8]:
# create timestamp udfs
get_hour = udf(lambda x: dt.datetime.fromtimestamp(x / 1000.0).hour)
get_year = udf(lambda x: dt.datetime.fromtimestamp(x / 1000.0).year)
get_month = udf(lambda x: dt.datetime.fromtimestamp(x / 1000.0).month)
get_day = udf(lambda x: dt.datetime.fromtimestamp(x / 1000.0).day)

In [9]:
selected_data = selected_data.withColumn('year',get_year(selected_data.ts))\
                            .withColumn('month',get_month(selected_data.ts))\
                            .withColumn('day',get_day(selected_data.ts))\
                            .withColumn('hour',get_hour(selected_data.ts))\

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [10]:
check_for_churn = udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, IntegerType())
check_for_downgrade_visit = udf(lambda x: 1 if x == 'Downgrade' else 0, IntegerType())
check_for_downgrade_submit = udf(lambda x: 1 if x == 'Submit Downgrade' else 0, IntegerType())
check_for_playlist = udf(lambda x: 1 if x == 'Add to Playlist' else 0, IntegerType())
check_for_friends = udf(lambda x: 1 if x == 'Add Friend' else 0, IntegerType())
check_for_adverts = udf(lambda x: 1 if x == 'Roll Advert' else 0, IntegerType())
check_for_logout = udf(lambda x: 1 if x == 'Logout' else 0, IntegerType())
# mark listening to song event
song_event = udf(lambda x: 1 if x == 'NextSong' else 0, IntegerType())
# mark listening to song event
check_level = udf(lambda x: 1 if x == 'paid' else 0, IntegerType())
check_thumbsup = udf(lambda x: 1 if x == 'Thumbs Up' else 0, IntegerType())
check_thumbsdown = udf(lambda x: 1 if x == 'Thumbs Down' else 0, IntegerType())

In [11]:
windowsum = Window.partitionBy("userId")

In [12]:
# calculate start and end time for each user in the database
selected_data = selected_data.withColumn('endDate',from_unixtime(maximum(selected_data.ts/1000.0).over(windowsum)))\
                             .withColumn('startDate',from_unixtime(minimum(selected_data.ts/1000.0).over(windowsum)))

# calculate duration per user
selected_data = selected_data\
        .withColumn('durationDays',datediff('endDate','startDate'))

In [13]:
# adding weekly timestamps to track behaviour over time
selected_data = selected_data.withColumn('weekBeforeEndDate', date_add(selected_data.endDate,-7))\
                                .withColumn('twoweekBeforeEndDate', date_add(selected_data.endDate,-14))\
                                .withColumn('threeweekBeforeEndDate', date_add(selected_data.endDate,-21))\
                                .withColumn('fourweekBeforeEndDate', date_add(selected_data.endDate,-28))

In [14]:
selected_data = selected_data.withColumn('churn',check_for_churn('page'))\
                            .withColumn('downgradeVisit',check_for_downgrade_visit('page'))\
                            .withColumn('downgradeSubmit',check_for_downgrade_submit('page'))\
                            .withColumn('addToPlaylist',check_for_playlist('page'))\
                            .withColumn('addFriend',check_for_friends('page'))\
                            .withColumn('rollAdvert',check_for_adverts('page'))\
                            .withColumn('logout',check_for_logout('page'))\
                            .withColumn('payingUser',check_level('level'))\
                            .withColumn('Up',check_thumbsup('page'))\
                            .withColumn('Down',check_thumbsdown('page'))\
                            .withColumn('listeningSongs',song_event(selected_data.page))

In [15]:

selected_data = selected_data.withColumn('churned',total('churn').over(windowsum))

In [16]:
selected_data = selected_data.withColumn('paying_when_churned',when((selected_data.churn == 1) \
                                                                    & (selected_data.payingUser == 1), 1)\
                                                                     .otherwise(0))
                                         

In [17]:
selected_data.select(['churn','paying_when_churned']).groupBy('churn').avg('paying_when_churned').show()

+-----+------------------------+
|churn|avg(paying_when_churned)|
+-----+------------------------+
|    1|      0.5961538461538461|
|    0|                     0.0|
+-----+------------------------+



In [18]:
selected_data.filter(selected_data.churn == 1)\
                    .groupBy('payingUser').count().show()

+----------+-----+
|payingUser|count|
+----------+-----+
|         1|   31|
|         0|   21|
+----------+-----+



In [19]:
selected_data.filter(from_unixtime(selected_data.ts/1000.0) == selected_data.endDate)\
                        .select(['userId','churned','payingUser']).dropDuplicates()\
                        .groupBy('churned').avg('payingUser').show()

+-------+------------------+
|churned|   avg(payingUser)|
+-------+------------------+
|      0|0.6589595375722543|
|      1|0.5961538461538461|
+-------+------------------+



In [20]:
# filter out last week of user interactions
selected_data_last_week = selected_data\
                            .filter(from_unixtime(selected_data.ts/1000.0) > selected_data.weekBeforeEndDate)
selected_data_before_last_week = selected_data\
                            .filter(from_unixtime(selected_data.ts/1000.0) < selected_data.weekBeforeEndDate)

In [21]:
def calculate_total_user_activity(df):
    # find how much time canceling users were subscribed
    windowsum = Window.partitionBy("userId")

    # mark all instances of users that canceled subscription
    df = df.withColumn('downgradeVisited',total('downgradeVisit').over(windowsum))\
                        .withColumn('downgradeSubmitted',total('downgradeSubmit').over(windowsum))\

    
    # calculate total songs listened per user
    df = df.withColumn('songsListened', total('listeningSongs').over(windowsum))
    df = df.withColumn('songsAddedToPlaylist', total('addToPlaylist').over(windowsum))
    df = df.withColumn('friendsAdded', total('addFriend').over(windowsum))
    df = df.withColumn('rolledAdverts', total('rollAdvert').over(windowsum))
    df = df.withColumn('logouts', total('logout').over(windowsum))
    df = df.withColumn('DownVoted', total('Down').over(windowsum))
    df = df.withColumn('UpVoted', total('Up').over(windowsum))
    df = df.withColumn('payingChurned', total('paying_when_churned').over(windowsum))
    
    return df

In [22]:
def calculate_days_of_actual_interaction(df):
    days_interacting_per_user = df.select('userId','year','month','day').dropDuplicates()\
                            .groupBy('userId').count().withColumnRenamed('count','daysInteracting')
    df = df.join(days_interacting_per_user,'userId','left')
    
    return df

In [23]:
def calculate_user_activity_per_day(df):
    # calculate average amount of songs listened per user
    df = df.withColumn('songsListenedPerDay'\
                                             , df.songsListened/df.daysInteracting)
    # calculate average amount of songs listened per user
    df = df.withColumn('songsAddedToPlaylistPerDay'\
                                             , df.songsAddedToPlaylist/df.daysInteracting)
    # calculate average amount of songs listened per user
    df = df.withColumn('friendsAddedPerDay'\
                                             , df.friendsAdded/df.daysInteracting)
    # calculate average amount of songs listened per user
    df = df.withColumn('rolledAdvertsPerDay'\
                                             , df.rolledAdverts/df.daysInteracting)
    # calculate average amount of songs listened per user
    df = df.withColumn('logoutsPerDay'\
                                             , df.logouts/df.daysInteracting)
    df = df.withColumn('upVotedPerDay'\
                                             , df.UpVoted/df.daysInteracting)
    df = df.withColumn('DownVotedPerDay'\
                                             , df.DownVoted/df.daysInteracting)
    df = df.withColumn('DownVotedPerSong'\
                                             , df.DownVoted/df.songsListened)
    df = df.withColumn('UpVotedPerSong'\
                                             , df.UpVoted/df.songsListened)
    df = df.withColumn('rolledAdvertsPerSong'\
                                             , df.rolledAdverts/df.songsListened)
    
    return df


In [24]:
selected_data = calculate_total_user_activity(selected_data)
selected_data_last_week = calculate_total_user_activity(selected_data_last_week)
selected_data_before_last_week = calculate_total_user_activity(selected_data_before_last_week)

In [25]:
selected_data = calculate_days_of_actual_interaction(selected_data)
selected_data_last_week = calculate_days_of_actual_interaction(selected_data_last_week)
selected_data_before_last_week = calculate_days_of_actual_interaction(selected_data_before_last_week)

In [26]:
selected_data = calculate_user_activity_per_day(selected_data)
selected_data_last_week = calculate_user_activity_per_day(selected_data_last_week)
selected_data_before_last_week = calculate_user_activity_per_day(selected_data_before_last_week)

In [27]:
selected_data = selected_data.withColumn('intensityOfInteraction',\
                                         selected_data.daysInteracting/selected_data.durationDays*100)\
                             .fillna(0,subset=['intensityOfInteraction'])

In [28]:
selected_data.count()

278154

In [29]:
selected_data.select('userId').dropDuplicates().count()

225

In [30]:
selected_data.select(['year','month','day']).dropDuplicates().count()

63

In [31]:
if True:
    selected_data_last_week.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('songsListenedPerDay'
                                                 , 'songsAddedToPlaylistPerDay'
                                                 , 'friendsAddedPerDay'
                                                 , 'rolledAdvertsPerDay').show()

+-------+------------------------+-------------------------------+-----------------------+------------------------+
|churned|avg(songsListenedPerDay)|avg(songsAddedToPlaylistPerDay)|avg(friendsAddedPerDay)|avg(rolledAdvertsPerDay)|
+-------+------------------------+-------------------------------+-----------------------+------------------------+
|      0|       67.96643270024774|             1.7996696944673825|     1.4228254885769334|       1.176520781723094|
|      1|       72.11620879120879|             2.0535714285714284|     1.2995421245421248|       2.235164835164835|
+-------+------------------------+-------------------------------+-----------------------+------------------------+



In [32]:
if True:
    selected_data_before_last_week.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('songsListenedPerDay'
                                                 , 'songsAddedToPlaylistPerDay'
                                                 , 'friendsAddedPerDay'
                                                 , 'rolledAdvertsPerDay').show()

+-------+------------------------+-------------------------------+-----------------------+------------------------+
|churned|avg(songsListenedPerDay)|avg(songsAddedToPlaylistPerDay)|avg(friendsAddedPerDay)|avg(rolledAdvertsPerDay)|
+-------+------------------------+-------------------------------+-----------------------+------------------------+
|      0|       62.07086336948415|             1.7436155580180257|      1.217066339866902|      1.4343073569314155|
|      1|       60.37457192442165|             1.6358783643802142|     1.1811804282622829|      2.0764543435622493|
+-------+------------------------+-------------------------------+-----------------------+------------------------+



In [33]:
if True:
    selected_data_last_week.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('logoutsPerDay'
                                                 , 'DownVotedPerSong'
                                                 , 'UpVotedPerSong'
                                                 , 'rolledAdvertsPerSong').show()

+-------+------------------+---------------------+--------------------+-------------------------+
|churned|avg(logoutsPerDay)|avg(DownVotedPerSong)| avg(UpVotedPerSong)|avg(rolledAdvertsPerSong)|
+-------+------------------+---------------------+--------------------+-------------------------+
|      0|1.0116226259289847| 0.009372752978787011|0.053240342811277756|      0.03133375640226847|
|      1|1.0345695970695972| 0.017472523157041432|0.056052836564706374|      0.05443498097071554|
+-------+------------------+---------------------+--------------------+-------------------------+



In [34]:
if True:
    selected_data_before_last_week.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('logoutsPerDay'
                                                 , 'DownVotedPerSong'
                                                 , 'UpVotedPerSong'
                                                 , 'rolledAdvertsPerSong').show()

+-------+------------------+---------------------+-------------------+-------------------------+
|churned|avg(logoutsPerDay)|avg(DownVotedPerSong)|avg(UpVotedPerSong)|avg(rolledAdvertsPerSong)|
+-------+------------------+---------------------+-------------------+-------------------------+
|      0|0.8941634838992512| 0.010758008401351918|0.05558031808778565|     0.035232856221563316|
|      1|0.9535670020616785|  0.01566800915504926|0.04463174051627814|      0.05176544836048677|
+-------+------------------+---------------------+-------------------+-------------------------+



In [35]:
# keep only distinct user calculations
selected_data_last_week = selected_data_last_week.select(['userId','songsListenedPerDay','songsAddedToPlaylistPerDay']).dropDuplicates()
selected_data_before_last_week = selected_data_before_last_week.select(['userId','songsListenedPerDay','songsAddedToPlaylistPerDay']).dropDuplicates()

selected_data_last_week = selected_data_last_week\
                            .withColumnRenamed('songsAddedToPlaylistPerDay','songsAddedToPlaylistPerDayLastWeek')\
                            .withColumnRenamed('songsListenedPerDay','songsListenedPerDayLastWeek')

selected_data_before_last_week = selected_data_before_last_week\
                            .withColumnRenamed('songsAddedToPlaylistPerDay','songsAddedToPlaylistPerDayBeforeLastWeek')\
                            .withColumnRenamed('songsListenedPerDay','songsListenedPerDayBeforeLastWeek')

In [36]:
selected_data = selected_data.join(selected_data_last_week\
                    .select(['userId'\
                             ,'songsAddedToPlaylistPerDayLastWeek'\
                             ,'songsListenedPerDayLastWeek']),'userId','left')

In [37]:
selected_data = selected_data.join(selected_data_before_last_week\
                    .select(['userId'\
                             ,'songsAddedToPlaylistPerDayBeforeLastWeek'\
                             ,'songsListenedPerDayBeforeLastWeek']),'userId','left')

In [38]:
selected_data = selected_data.withColumn('playlistActivityChange'\
                                          ,selected_data.songsAddedToPlaylistPerDayLastWeek/selected_data.songsAddedToPlaylistPerDayBeforeLastWeek)\
                             .withColumn('listeningActivityChange'\
                                          ,selected_data.songsListenedPerDayLastWeek/selected_data.songsListenedPerDayBeforeLastWeek)

In [39]:
if True:
    selected_data.select(['userId','churned', 'intensityOfInteraction'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('intensityOfInteraction').show()

+-------+---------------------------+
|churned|avg(intensityOfInteraction)|
+-------+---------------------------+
|      0|         30.525659485236947|
|      1|          51.30873352861251|
+-------+---------------------------+



In [40]:
if True:
    selected_data.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('songsListenedPerDay'
                                                 , 'songsAddedToPlaylistPerDay'
                                                 , 'friendsAddedPerDay'
                                                 , 'rolledAdvertsPerDay').show()

+-------+------------------------+-------------------------------+-----------------------+------------------------+
|churned|avg(songsListenedPerDay)|avg(songsAddedToPlaylistPerDay)|avg(friendsAddedPerDay)|avg(rolledAdvertsPerDay)|
+-------+------------------------+-------------------------------+-----------------------+------------------------+
|      0|       63.75151337240347|             1.7669149523780139|     1.2906602801681264|       1.425920387136379|
|      1|       61.28465427752686|             1.7620385576687174|     1.1802959788202787|        2.30370850481346|
+-------+------------------------+-------------------------------+-----------------------+------------------------+



In [41]:
if True:
    selected_data.select(['userId','churned','songsListenedPerDay'\
                                             , 'songsAddedToPlaylistPerDay'
                                             , 'friendsAddedPerDay'
                                             , 'rolledAdvertsPerDay'
                                             , 'logoutsPerDay'
                                             , 'upVotedPerDay'
                                             , 'DownVotedPerDay'
                                             , 'DownVotedPerSong'
                                             , 'UpVotedPerSong'
                                             , 'rolledAdvertsPerSong'])\
                                            .dropDuplicates()\
                                            .groupBy('churned')\
                                            .avg('logoutsPerDay'
                                                 , 'DownVotedPerSong'
                                                 , 'UpVotedPerSong'
                                                 , 'rolledAdvertsPerSong').show()

+-------+------------------+---------------------+-------------------+-------------------------+
|churned|avg(logoutsPerDay)|avg(DownVotedPerSong)|avg(UpVotedPerSong)|avg(rolledAdvertsPerSong)|
+-------+------------------+---------------------+-------------------+-------------------------+
|      0|0.9333672364197397|  0.01020712194202562|0.05509142892317728|      0.03583661330296051|
|      1|0.9926640147325367|  0.01751018776131704|0.05363922514672142|      0.05866835308958933|
+-------+------------------+---------------------+-------------------+-------------------------+



In [42]:
selected_data.select(['userId','payingChurned']).dropDuplicates().groupBy('paying')

In [43]:
selected_data_distinct_users = selected_data.select(['userId'\
                                                     ,'churned'\
                                                     ,'songsListenedPerDay'\
                                                     ,'DownVotedPerSong'\
                                                     ,'friendsAddedPerDay'\
                                                     ,'intensityOfInteraction'\
                                                     ,'playlistActivityChange'\
                                                     ,'listeningActivityChange']).dropDuplicates()

In [44]:
# calculate means to replace nulls
songsListenedPerDayMean = 0#selected_data_distinct_users.select(mean('songsListenedPerDay')).collect()[0][0]
DownVotedPerSongMean = 0#selected_data_distinct_users.select(mean('DownVotedPerSong')).collect()[0][0]
friendsAddedPerDayMean = 0#selected_data_distinct_users.select(mean('friendsAddedPerDay')).collect()[0][0]
intensityOfInteractionMean = 0#selected_data_distinct_users.select(mean('intensityOfInteraction')).collect()[0][0]
playlistActivityChangeMean = 0#selected_data_distinct_users.select(mean('playlistActivityChange')).collect()[0][0]
listeningActivityChangeMean = 0#selected_data_distinct_users.select(mean('listeningActivityChange')).collect()[0][0]


In [45]:
selected_data_distinct_users = selected_data_distinct_users.fillna(0,subset=['songsListenedPerDay'\
                                                     ,'DownVotedPerSong'\
                                                     ,'friendsAddedPerDay'\
                                                     ,'intensityOfInteraction'\
                                                     ,'playlistActivityChange'\
                                                     ,'listeningActivityChange'])

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [46]:
data = selected_data_distinct_users.select(['churned'\
                                                     ,'songsListenedPerDay'\
                                                     ,'DownVotedPerSong'\
                                                     ,'friendsAddedPerDay'\
                                                     ,'intensityOfInteraction'\
                                                     ,'playlistActivityChange'\
                                                     ,'listeningActivityChange'])\
                                   .withColumnRenamed('churned','label')

In [47]:
assembler = VectorAssembler(inputCols=["songsListenedPerDay"\
                                       , "DownVotedPerSong"
                                       , 'friendsAddedPerDay'
                                       , 'intensityOfInteraction'
                                       , 'playlistActivityChange'
                                       ,'listeningActivityChange'], outputCol="features")

In [48]:
data = assembler.transform(data)

In [49]:
data = data.select(['label','features'])

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [50]:
# Split data into training and testing sets
trainingData, testData = data.randomSplit([0.7, 0.3], seed=12345)

In [51]:
# instanciate the linear regression model
lr = LogisticRegression(maxIter=5, regParam=0.0)
dtr = DecisionTreeClassifier(labelCol="label", featuresCol="features")
#rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [52]:
# Define a grid of hyperparameters to search over
paramGrid = ParamGridBuilder() \
    .addGrid(dtr.maxDepth, [4, 6]) \
    .build()

In [53]:
# Evaluate the predictions using F1 score
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# Create a CrossValidator to perform grid search
cv = CrossValidator(estimator=dtr, estimatorParamMaps=paramGrid, evaluator=evaluator)

In [ ]:
# Fit the CrossValidator to the training data
dt_cvModel = cv.fit(trainingData)

In [ ]:
dt_model = dtr.fit(trainingData)

In [ ]:
#rf_model = rf.fit(trainingData)

In [ ]:
lr_model = lr.fit(trainingData)

In [ ]:
# use test data to make predictions
#rf_predictions = rf_model.transform(testData)
# use test data to make predictions
lr_predictions = lr_model.transform(testData)
# Evaluate the best model on the test data
#cv_predictions = dt_cvModel.transform(testData)
# Evaluate the best model on the test data
dt_predictions = dt_model.transform(testData)

In [ ]:
dt_f1_score = evaluator.evaluate(dt_predictions)
lr_f1_score = evaluator.evaluate(lr_predictions)
#cv_f1_score = evaluator.evaluate(cv_predictions)

In [ ]:
print(dt_f1_score)
print(lr_f1_score)
#print(cv_f1_score)

In [ ]:
#print("Best maxDepth: ", dt_cvModel.bestModel.getMaxDepth())

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.